# Airport IAT

## Import libraries

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
import psycopg2
import requests
from zipfile import *
from sql_functions import *

## import url, station, date etc.

In [ ]:
url = "https://meteostat.p.rapidapi.com/stations/daily"

querystring = {"station":"72243","start":"2017-08-01","end":"2017-08-31"}

headers = {
	"X-RapidAPI-Key": "6e5bd50375msh22713539588b39bp1c509djsn30f4d6bf3d1f",
	"X-RapidAPI-Host": "meteostat.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

## print headers

In [ ]:
response.headers

## print cookies

In [ ]:
response.cookies

## print encoding

In [ ]:
response.encoding

## print content

In [ ]:
response.content

## Apply JSON decoder and save in weather_dfw

In [ ]:
weather_iat = response.json()
print(weather_iat)

## import json, create json object, print json_object

In [ ]:
json_object = json.loads(response.content)
json.dumps(json_object, indent = 3)

## print type

In [ ]:
type(weather_iat)

## Loop through and print key-value pairs

In [ ]:
for key, value in weather_iat.items():
    print(key + ':', value)

## print content of json_object

In [ ]:
json.dumps(json_object, indent = 3)

## Flatten json and save in weather_iat_norm

In [ ]:
weather_iat_norm = pd.json_normalize(weather_iat)
weather_iat_norm

## advanced flattening with json_normalize()

In [ ]:
weather_iat_df = pd.json_normalize(weather_iat, 
                                  record_path="data", 
                                  errors='ignore')

weather_iat_df

## change format of date

In [ ]:
weather_iat_df['date'] = pd.to_datetime(weather_iat_df['date'], format="%Y/%m/%d")

## push the data to a table in DBeaver

In [ ]:
schema = 'muc_analytics_22_1'
engine = get_engine()

table_name = 'weather_iat_g5'

if engine!=None:
    try:
        weather_iat_df.to_sql(name=table_name,
                        con=engine,
                        if_exists='replace',
                        schema=schema, 
                        index=False,
                        chunksize=5000, 
                        method='multi')
        print(f"The {table_name} table was imported successfully.")
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None